In [1]:
# Libraries
import math, re, os
from kaggle_secrets import UserSecretsClient
from kaggle_datasets import KaggleDatasets
import json
import shutil
import zipfile
import re
import tensorflow as tf

# EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from joblib import Parallel, delayed

# Data
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

E0000 00:00:1724566053.521199    1791 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0825 06:07:33.529642077    1791 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0825 06:07:33.529659274    1791 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0825 06:07:33.529663242    1791 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0825 06:07:33.529666116    1791 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0825 06:07:33.529669006    1791 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

Tensorflow version 2.16.1


In [2]:
AUTO = tf.data.experimental.AUTOTUNE

# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1724566059.838643    1791 service.cc:145] XLA service 0x5c5c6292a8d0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724566059.838705    1791 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1724566059.838710    1791 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1724566059.838713    1791 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1724566059.838716    1791 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1724566059.838718    1791 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1724566059.838721    1791 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1724566059.838724    1791 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1724566059.838726    1791 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8


In [3]:
!pip install --upgrade kaggle


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
# Kaggle API Using Secrets

from kaggle_secrets import UserSecretsClient

# Brought key info
user_secrets = UserSecretsClient()
kaggle_key = user_secrets.get_secret("jiyoungroh")

# Setting
os.environ['KAGGLE_KEY'] = kaggle_key
os.environ['KAGGLE_USERNAME'] = "jiyoungroh"

In [5]:
!kaggle competitions download -c playground-series-s4e8

playground-series-s4e8.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
# Unzip
with zipfile.ZipFile('/kaggle/working/playground-series-s4e8.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/')

# Loading Data
train_data = pd.read_csv('/kaggle/working/train.csv')
test_data = pd.read_csv('/kaggle/working/test.csv')
sample_submission = pd.read_csv('/kaggle/working/sample_submission.csv')

In [7]:
pd.set_option('display.max_columns', None)
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


# 1. Data Preprocessing

1. 수치형 변수들 로그화 (나중에 성능 안 좋으면 기존 변수로 사용해보기)
2. Stem-width와 Cap-diameter 처리 (PCA) or 모델 학습할 때 정규화나 교차 검증해서 과적합 방지하기 (PCA 먼저 해볼거임)
3. 범주형 변수들 random forest나 lasso, chi-square 등 사용해서 feature selection 할 예정
4. 모델링 한 다음에 성능 보고 위에 과정들 중 선택해서 다시 해보기


## 1. lOG

In [8]:
# Feature Setting

train_numeric_columns =  train_data[['cap-diameter', 'stem-height', 'stem-width']]

train_categoric_columns = train_data[['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
    'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root',
    'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']]                                 

train_remove_categoric_columns = train_data[[
    'cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
    'gill-attachment', 'gill-spacing', 'gill-color',
    'stem-surface', 'stem-color', 'has-ring', 'ring-type', 'habitat', 'season'
]]
                                

In [9]:
# Log Numeric
train_data['log_cap-diameter'] = np.log1p(train_data['cap-diameter'])
train_data['log_stem-height'] = np.log1p(train_data['stem-height'])
train_data['log_stem-width'] = np.log1p(train_data['stem-width'])

## 2. Remove or Replace Missing Value

### 1. Numeric Data

Remove or Replace Missing Value in Numeric (average or mean..)

Remove - cap-diameter, stem-width

Replace - Median(before log), Mean(after log) - stem-height


In [10]:
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,log_cap-diameter,log_stem-height,log_stem-width
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a,2.282382,1.706565,2.796671
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w,1.706565,1.756132,2.012233
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w,2.071913,2.060514,2.391511
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u,1.585145,1.640937,2.018895
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a,1.924249,1.474763,2.236445


In [11]:
# Remove numeric NA from train data
train_data = train_data.dropna(subset=['cap-diameter', 'stem-width'])

In [12]:
# Replace stem-height NA to median value
train_data['stem-height'].fillna(train_data['stem-height'].median(), inplace=True)

In [13]:
# Replace log stem-height NA to mean value
train_data['log_stem-height'].fillna(train_data['log_stem-height'].mean(), inplace=True)

## 2. Categoric Data

Remove NaN: cap-shape, cap-color, does-bruise-or-bleed, gil-color, stem-color, has-ring, habitat, season

KNN Imputation: cap-surface, gill-attachment, gill-spacing, stem-surface, ring-type

Remove Column: stem-root, veil-type, veil-color, spore-print-color

In [14]:
# Remove Na in train data 
train_data = train_data.dropna(subset=['cap-shape', 'cap-color', 'does-bruise-or-bleed', 
                                       'gill-color', 'stem-color', 'has-ring', 
                                       'habitat', 'season'])

In [15]:
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,log_cap-diameter,log_stem-height,log_stem-width
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a,2.282382,1.706565,2.796671
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w,1.706565,1.756132,2.012233
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w,2.071913,2.060514,2.391511
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u,1.585145,1.640937,2.018895
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a,1.924249,1.474763,2.236445


In [16]:
for col in ['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-surface', 'ring-type']:
    le = LabelEncoder()
    combined_data = pd.concat([train_data[col]], axis=0).astype(str)
    le.fit(combined_data)
    train_data[col] = le.transform(train_data[col].astype(str))

In [17]:
# KNN Imputer reset(k=5 is basic)
imputer = KNNImputer(n_neighbors=5)

# KNN Imputation run (cap-surface, gill-attachment, gill-spacing, stem-surface, ring-type)
train_data[['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-surface', 'ring-type']] = imputer.fit_transform(
    train_data[['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-surface', 'ring-type']]
)

In [18]:
# Remove Column (stem-root, veil-type, veil-color, spore-print-color)
train_data = train_data.drop(columns=['stem-root', 'veil-type', 'veil-color', 'spore-print-color'])

In [19]:
print(train_data.isna().sum())

id                      0
class                   0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-surface            0
stem-color              0
has-ring                0
ring-type               0
habitat                 0
season                  0
log_cap-diameter        0
log_stem-height         0
log_stem-width          0
dtype: int64


## 2. PCA

In [20]:
# Data Prepration for PCA
PCA_numeric_data = train_data[['cap-diameter', 'stem-width']].values
PCA_log_numeric_data = train_data[['log_cap-diameter', 'log_stem-width']].values

# PCA Model
pca = PCA(n_components=1)  # Decrease 1 components
pca_result = pca.fit_transform(PCA_numeric_data)

# Add Result of PCA to new column
train_data['pca_cap_stem'] = pca_result

In [21]:
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season,log_cap-diameter,log_stem-height,log_stem-width,pca_cap_stem
0,0,e,8.80,f,73.0,u,f,43.0,28.0,w,4.51,15.39,48.0,w,f,18.0,d,a,2.282382,1.706565,2.796671,4.899435
1,1,p,4.51,x,56.0,o,f,43.0,28.0,n,4.79,6.48,59.0,o,t,40.0,d,w,1.706565,1.756132,2.012233,-4.989392
2,2,e,6.94,f,73.0,b,f,75.0,28.0,w,6.85,9.93,52.0,n,f,18.0,l,w,2.071913,2.060514,2.391511,-0.822813
3,3,e,3.88,f,82.0,g,f,70.0,41.0,g,4.16,6.53,48.0,w,f,18.0,d,u,1.585145,1.640937,2.018895,-5.221063
4,4,e,5.85,x,65.0,w,f,46.0,41.0,w,3.37,8.36,48.0,w,f,18.0,g,a,1.924249,1.474763,2.236445,-2.711960


## 3. Feature Selection

1. Filter Method
2. Wrapper Method
3. Embedded Method

In [22]:
# Filter Method
selected_features = [
    'cap-surface', 'gill-color', 'stem-color', 'habitat','stem-surface', 'pca_cap_stem']

In [23]:
def chunked_one_hot_encoding(data, column, chunk_size=50000):
    num_chunks = len(data) // chunk_size + 1
    encoder = OneHotEncoder(sparse_output=False, dtype=int)
    
    # 구간별로 인코딩된 데이터 저장
    encoded_data = []
    
    for i in range(num_chunks):
        chunk = data.iloc[i*chunk_size:(i+1)*chunk_size]
        encoded_chunk = encoder.fit_transform(chunk[[column]])
        encoded_df = pd.DataFrame(encoded_chunk, columns=[f"{column}_{cat}" for cat in encoder.categories_[0]])
        encoded_data.append(encoded_df.reset_index(drop=True))
    
    # 구간별로 인코딩된 데이터를 합침
    return pd.concat(encoded_data, axis=0).reset_index(drop=True)

# class 변수에 대해 원 핫 인코딩 수행
encoded_class = chunked_one_hot_encoding(train_data, 'class')

# 원래 데이터에서 class 열을 제거하고 인코딩된 class 열을 추가
train_data = train_data.drop(columns=['class']).reset_index(drop=True)
train_data = pd.concat([train_data, encoded_class], axis=1)

In [24]:
train_data.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season,log_cap-diameter,log_stem-height,log_stem-width,pca_cap_stem,class_e,class_p
0,0,8.80,f,73.0,u,f,43.0,28.0,w,4.51,15.39,48.0,w,f,18.0,d,a,2.282382,1.706565,2.796671,4.899435,1,0
1,1,4.51,x,56.0,o,f,43.0,28.0,n,4.79,6.48,59.0,o,t,40.0,d,w,1.706565,1.756132,2.012233,-4.989392,0,1
2,2,6.94,f,73.0,b,f,75.0,28.0,w,6.85,9.93,52.0,n,f,18.0,l,w,2.071913,2.060514,2.391511,-0.822813,1,0
3,3,3.88,f,82.0,g,f,70.0,41.0,g,4.16,6.53,48.0,w,f,18.0,d,u,1.585145,1.640937,2.018895,-5.221063,1,0
4,4,5.85,x,65.0,w,f,46.0,41.0,w,3.37,8.36,48.0,w,f,18.0,g,a,1.924249,1.474763,2.236445,-2.711960,1,0


In [25]:
def chunked_target_encoding(train_data, test_data, target, col, chunk_size=50000, n_splits=5):
    num_chunks = len(train_data) // chunk_size + 1
    mean_target = train_data[target].mean()
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    train_encoded_full = np.zeros(train_data.shape[0])
    test_encoded_full = np.zeros(test_data.shape[0])

    for i in range(num_chunks):
        train_chunk = train_data.iloc[i*chunk_size:(i+1)*chunk_size].copy()
        test_chunk = test_data.copy()

        train_encoded = np.zeros(train_chunk.shape[0])
        test_encoded = np.zeros(test_chunk.shape[0])

        for train_index, val_index in kf.split(train_chunk):
            train_fold, val_fold = train_chunk.iloc[train_index], train_chunk.iloc[val_index]
            means = train_fold.groupby(col)[target].mean()
            
            train_encoded[val_index] = val_fold[col].map(means).fillna(mean_target)
            test_encoded += test_chunk[col].map(means).fillna(mean_target) / n_splits

        train_encoded_full[i*chunk_size:(i+1)*chunk_size] = train_encoded
        test_encoded_full += test_encoded / num_chunks

    return train_encoded_full, test_encoded_full

# 타겟 인코딩 적용
categorical_features = ['cap-surface', 'gill-color', 'stem-color', 'habitat', 'stem-surface']
for col in categorical_features:
    train_data[col + '_encoded'], test_data[col + '_encoded'] = chunked_target_encoding(train_data, test_data, 'class_e', col)

# 인코딩된 변수들만 사용하도록 업데이트
encoded_features = [col + '_encoded' for col in categorical_features] + ['pca_cap_stem']

In [26]:
train_data.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season,log_cap-diameter,log_stem-height,log_stem-width,pca_cap_stem,class_e,class_p,cap-surface_encoded,gill-color_encoded,stem-color_encoded,habitat_encoded,stem-surface_encoded
0,0,8.80,f,73.0,u,f,43.0,28.0,w,4.51,15.39,48.0,w,f,18.0,d,a,2.282382,1.706565,2.796671,4.899435,1,0,0.632735,0.565330,0.576750,0.470406,0.485669
1,1,4.51,x,56.0,o,f,43.0,28.0,n,4.79,6.48,59.0,o,t,40.0,d,w,1.706565,1.756132,2.012233,-4.989392,0,1,0.529051,0.284326,0.399862,0.470406,0.304646
2,2,6.94,f,73.0,b,f,75.0,28.0,w,6.85,9.93,52.0,n,f,18.0,l,w,2.071913,2.060514,2.391511,-0.822813,1,0,0.625532,0.565320,0.391055,0.610882,0.604030
3,3,3.88,f,82.0,g,f,70.0,41.0,g,4.16,6.53,48.0,w,f,18.0,d,u,1.585145,1.640937,2.018895,-5.221063,1,0,0.530814,0.507954,0.572575,0.466357,0.483924
4,4,5.85,x,65.0,w,f,46.0,41.0,w,3.37,8.36,48.0,w,f,18.0,g,a,1.924249,1.474763,2.236445,-2.711960,1,0,0.559783,0.561877,0.573767,0.330876,0.483230


In [27]:
selected_features = [
    'cap-surface_encoded', 'gill-color_encoded', 'stem-color_encoded', 'habitat_encoded','stem-surface_encoded', 'pca_cap_stem']

In [28]:
# 80:20 비율로 train과 test 데이터셋 나누기
X_train, X_test, y_train, y_test = train_test_split(train_data[selected_features], train_data[['class_e', 'class_p']], test_size=0.2, random_state=42)

# 4. Modeling

## 4-1. Basic Modeling (Logistic Regression, Random Forest, XGBoost)

In [ ]:
# 예측 결과 CSV 파일을 불러오기
predictions = pd.read_csv('/kaggle/input/your-dataset-name/model_predictions.csv')

# 평가 지표 CSV 파일을 불러오기
metrics = pd.read_csv('/kaggle/input/your-dataset-name/model_metrics.csv')

# 데이터 확인
print(predictions.head())
print(metrics.head())

### 1. Logistic Regression

In [29]:
with strategy.scope():
    model_lr = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model_lr.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history_lr = model_lr.fit(X_train, y_train['class_e'], epochs=10, batch_size=1024, validation_split=0.2)

I0000 00:00:1724566428.945005    1791 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1/10


2024-08-25 06:13:52.269765: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1724566432.544207    2564 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(9e8a6b07c3a92615:0:0), session_name()


  21/1948 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.4530 - loss: 0.8724 

I0000 00:00:1724566432.881141    2564 tpu_compile_op_common.cc:245] Compilation of 9e8a6b07c3a92615:0:0 with session name  took 336.878367ms and succeeded
I0000 00:00:1724566432.885326    2564 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(9e8a6b07c3a92615:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_53797433552513609", property.function_library_fingerprint = 1079265638941637160, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "128,6,;128,1,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1724566432.885360    2564 tpu_compilation_cache_interface.cc:541] After adding entry for key 9e8a6b07

...

1948/1948 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6259 - loss: 0.6449

I0000 00:00:1724566446.463102    2610 tpu_compile_op_common.cc:245] Compilation of 7c848efdd4235929:0:0 with session name  took 305.799004ms and succeeded
I0000 00:00:1724566446.465774    2610 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(7c848efdd4235929:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_53797433552513609", property.function_library_fingerprint = 1079265638941637160, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "122,6,;122,1,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1724566446.465797    2610 tpu_compilation_cache_interface.cc:541] After adding entry for key 7c848efd

1948/1948 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.6259 - loss: 0.6449 - val_accuracy: 0.6994 - val_loss: 0.5670
Epoch 2/10


I0000 00:00:1724566450.609343    2577 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(374536a41a3e31e5:0:0), session_name()
I0000 00:00:1724566450.745828    2577 tpu_compile_op_common.cc:245] Compilation of 374536a41a3e31e5:0:0 with session name  took 136.447668ms and succeeded
I0000 00:00:1724566450.747053    2577 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(374536a41a3e31e5:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_9757088990094677515", property.function_library_fingerprint = 18423573722654458006, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "127,6,;127,1,;", property.guaranteed_constants_size = 0, embedding

1948/1948 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.7085 - loss: 0.5599 - val_accuracy: 0.7113 - val_loss: 0.5466
Epoch 3/10
1948/1948 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.7161 - loss: 0.5450 - val_accuracy: 0.7238 - val_loss: 0.5367
Epoch 4/10
1948/1948 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.7179 - loss: 0.5364 - val_accuracy: 0.7191 - val_loss: 0.5292
Epoch 5/10
1948/1948 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.7200 - loss: 0.5280 - val_accuracy: 0.7202 - val_loss: 0.5270
Epoch 6/10
1948/1948 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.7227 - loss: 0.5202 - val_accuracy: 0.7258 - val_loss: 0.5144
Epoch 7/10
1948/1948 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.7268 - loss: 0.5121 - val_accuracy: 0.7241 - val_loss: 0.5135
Epoch 8/10
1948/1948 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.7327 - loss: 0.5052 - val_accuracy: 0.7376 - val_loss: 0.5000
Epoch 9/10
1948/1948 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.7387 - loss: 0.4978 - val

### 2. Random Forest

In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model_rf = RandomForestClassifier(warm_start=True, random_state=42)
n_trees = 10  # Total number of trees
step = 1  # Number of trees to add at each iteration
accuracies = []

for i in range(step, n_trees + step, step):
    model_rf.set_params(n_estimators=i)
    model_rf.fit(X_train, y_train['class_e'])

    preds_rf = model_rf.predict(X_train)
    acc_rf = accuracy_score(y_train['class_e'], preds_rf)
    accuracies.append(acc_rf)
    print(f"Number of Trees: {i}, Accuracy: {acc_rf:.4f}")

# Final model fit
model_rf.fit(X_train, y_train['class_e'])

Number of Trees: 1, Accuracy: 0.9759
Number of Trees: 2, Accuracy: 0.9757
Number of Trees: 3, Accuracy: 0.9876
Number of Trees: 4, Accuracy: 0.9876
Number of Trees: 5, Accuracy: 0.9916
Number of Trees: 6, Accuracy: 0.9916
Number of Trees: 7, Accuracy: 0.9937
Number of Trees: 8, Accuracy: 0.9937
Number of Trees: 9, Accuracy: 0.9951
Number of Trees: 10, Accuracy: 0.9951


RandomForestClassifier(n_estimators=10, random_state=42, warm_start=True)

### 3. XG Boost

In [31]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 5.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [54]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(n_estimators=100, use_label_encoder=False)
model_xgb.fit(X_train, y_train['class_e'])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

### Save Model

In [57]:
preds_rf = model_rf.predict(X_test)
preds_xgb = model_xgb.predict(X_test)
preds_lr = model_lr.predict(X_test).flatten()  # 로지스틱 회귀는 확률로 예측하므로 flatten 후 이진값으로 변환
preds_lr_binary = (preds_lr > 0.5).astype(int)

# 결과를 CSV 파일로 저장
pd.DataFrame({
    'RandomForest': preds_rf,
    'XGBoost': preds_xgb,
    'LogisticRegression': preds_lr_binary
}).to_csv('LR_RF_XG_model_predictions.csv', index=False)

19480/19480 ━━━━━━━━━━━━━━━━━━━━ 548s 28ms/step


In [59]:
# Ensure y_test is a binary vector
y_test_binary = y_test['class_e']  # Assuming 'class_e' is the column you're predicting

# Calculate metrics
acc_rf = accuracy_score(y_test_binary, preds_rf)
f1_rf = f1_score(y_test_binary, preds_rf)

acc_xgb = accuracy_score(y_test_binary, preds_xgb)
f1_xgb = f1_score(y_test_binary, preds_xgb)

acc_lr = accuracy_score(y_test_binary, preds_lr_binary)
f1_lr = f1_score(y_test_binary, preds_lr_binary)

# Store metrics
metrics = {
    'Model': ['RandomForest', 'XGBoost', 'LogisticRegression'],
    'Accuracy': [acc_rf, acc_xgb, acc_lr],
    'F1 Score': [f1_rf, f1_xgb, f1_lr]
}

# Save to CSV
pd.DataFrame(metrics).to_csv('/kaggle/working/LR_RF_XG_model_metrics.csv', index=False)

In [60]:
file_path = '/kaggle/working/LR_RF_XG_model_metrics.csv'

# CSV 파일 읽기
metrics_df = pd.read_csv(file_path)

# 파일 내용 출력
print(metrics_df)

                Model  Accuracy  F1 Score
0        RandomForest  0.947214  0.941426
1             XGBoost  0.943508  0.937853
2  LogisticRegression  0.744830  0.715786
